# Тестовое задание

In [45]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import string

In [46]:
count_vect = CountVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
tfidf_vect = TfidfVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
porter_stemmer = nltk.stem.PorterStemmer()
tok = count_vect.build_tokenizer()
# tok = tfidf_vect.build_tokenizer()
# tok = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize
clf = LogisticRegression(solver='liblinear', class_weight = {0:4, 1:1}, penalty='l2')

## Cкачиваем файл с данными

In [47]:
# !wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
#или
#!wget https://drive.google.com/open?id=1DwwTyIn5zW7KTxp4bONHtKNa6DlT9PUj

## Если необходимо, преобразуем JSON к CSV

In [48]:
# df = pd.read_json ('Electronics_5.json', lines=True )
# df.to_csv ('Electronics_5.csv', index = None)

# Предобработка

In [49]:
dat = pd.read_csv("Electronics_5.csv", engine= 'python', error_bad_lines=False)[["asin","reviewText", "overall"]]

Skipping line 1609665: unexpected end of data


## Добавляем бинрную оценку

In [50]:
dat["bin"] = dat.overall > 3

## Отрежем кусок даты

In [51]:
dat_t, dat = dat, dat[:16000]

In [52]:
train_dat, privat_dat, train_ans, privat_ans = train_test_split(
    dat.reviewText, dat.bin, test_size=0.15)

In [53]:
train_dat, test_dat, train_ans, test_ans = train_test_split(
    train_dat, train_ans, test_size=0.2)

## Натравим на необработанные данные

In [54]:
count_vect.fit(dat.reviewText.apply(lambda x: np.str_(x)))
tfidf_vect.fit(dat.reviewText.apply(lambda x: np.str_(x)))
count_train_mtrx_dft = count_vect.transform(train_dat.apply(lambda x: np.str_(x)))
tfidf_train_mtrx_dft = tfidf_vect.transform(train_dat.apply(lambda x: np.str_(x)))
count_test_mtrx_dft = count_vect.transform(test_dat.apply(lambda x: np.str_(x)))
tfidf_test_mtrx_dft = tfidf_vect.transform(test_dat.apply(lambda x: np.str_(x)))

## Приводим к базовой форме

In [55]:
train_dat_tr = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), train_dat.apply(lambda x: np.str_(x)))))
test_dat_tr = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), test_dat.apply(lambda x: np.str_(x)))))
privat_dat_tr = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), privat_dat.apply(lambda x: np.str_(x)))))
dat["words"] = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), dat.reviewText.apply(lambda x: np.str_(x)))))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


## Натравим на обработанные данные

In [56]:
# count_vect.fit(dat.words)
# tfidf_vect.fit(dat.words)
count_train_mtrx = count_vect.transform(train_dat_tr)
tfidf_train_mtrx = tfidf_vect.transform(train_dat_tr)
count_test_mtrx = count_vect.transform(test_dat_tr)
tfidf_test_mtrx = tfidf_vect.transform(test_dat_tr)

## Проверим, что получилось

In [57]:
dat

,asin,reviewText,overall,bin,words
0,0528881469,We got this GPS for my husband who is an (OTR)...,5,True,we got thi gp for my husband who is an otr ove...
1,0528881469,"I'm a professional OTR truck driver, and I bou...",1,False,profession otr truck driver and bought tnd 700...
2,0528881469,"Well, what can I say. I've had this unit in m...",3,False,well what can say ve had thi unit in my truck ...
3,0528881469,"Not going to write a long review, even thought...",2,False,not go to write long review even thought thi u...
4,0528881469,I've had mine for a year and here's what we go...,1,False,ve had mine for year and here what we got it t...
...,...,...,...,...,...
15995,B00004ZCJE,I could not ask more of the Tiffen 58mm UV Pro...,5,True,could not ask more of the tiffen 58mm uv prote...
15996,B00004ZCJE,I ordered 2 Tiffen 52 mm UV filters for the 2 ...,4,True,order tiffen 52 mm uv filter for the lens that...
15997,B00004ZCJE,It protects the lens. But I still have glare f...,4,True,it protect the len but still have glare from l...
15998,B00004ZCJE,"buena calidad, resistente, producto bien hecho...",5,True,buena calidad resistent producto bien hecho la...


# Обучим модели и посмотрим их точность

## Обработанные с оптимальными параметрами

In [58]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2'],
    'class_weight': ['balanced', None, {0:4, 1:1}, {0:1, 1:1}]
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(count_train_mtrx, train_ans)
print(search.best_params_)

{'C': 0.2, 'class_weight': None, 'penalty': 'l2'}


In [59]:
accuracy_score(test_ans, search.best_estimator_.predict(count_test_mtrx))

0.8558823529411764

In [60]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2'],
    'class_weight': ['balanced', None, {0:4, 1:1}, {0:1, 1:1}]
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(tfidf_train_mtrx,  train_ans)
print(search.best_params_)

{'C': 2.5000000000000004, 'class_weight': None, 'penalty': 'l1'}


In [61]:
accuracy_score(test_ans, search.best_estimator_.predict(tfidf_test_mtrx))

0.8636029411764706

## Дефолтные с оптимальными параметрами

In [62]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2'],
    'class_weight': ['balanced', None, {0:4, 1:1}, {0:1, 1:1}]
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(count_train_mtrx_dft, train_ans)
print(search.best_params_)

{'C': 0.2, 'class_weight': None, 'penalty': 'l2'}


In [63]:
print(accuracy_score(test_ans, search.best_estimator_.predict(count_test_mtrx_dft)))
print(classification_report(test_ans, search.best_estimator_.predict(count_test_mtrx_dft)))

0.8716911764705882
              precision    recall  f1-score   support

       False       0.73      0.47      0.57       494
        True       0.89      0.96      0.92      2226

    accuracy                           0.87      2720
   macro avg       0.81      0.71      0.75      2720
weighted avg       0.86      0.87      0.86      2720



In [64]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2'],
    'class_weight': ['balanced', None, {0:4, 1:1}, {0:1, 1:1}]
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(tfidf_train_mtrx_dft, train_ans)
print(search.best_params_)

{'C': 2.9000000000000004, 'class_weight': None, 'penalty': 'l1'}


In [65]:
print(accuracy_score(test_ans, search.best_estimator_.predict(tfidf_test_mtrx_dft)))
print(classification_report(test_ans, search.best_estimator_.predict(tfidf_test_mtrx_dft)))


0.8738970588235294
              precision    recall  f1-score   support

       False       0.74      0.47      0.58       494
        True       0.89      0.96      0.93      2226

    accuracy                           0.87      2720
   macro avg       0.82      0.72      0.75      2720
weighted avg       0.86      0.87      0.86      2720



# Лучший результат у tfidf на необработанных данных

In [66]:
clf = LogisticRegression(solver='liblinear', class_weight = {0:4, 1:1}, C=2.6, penalty='l2')
clf.fit(tfidf_train_mtrx_dft, train_ans)
pred = clf.predict(tfidf_vect.transform(privat_dat.apply(lambda x: np.str_(x))))
print(accuracy_score(pred, privat_ans))
print(classification_report(pred, privat_ans))

0.8670833333333333
              precision    recall  f1-score   support

       False       0.64      0.63      0.63       435
        True       0.92      0.92      0.92      1965

    accuracy                           0.87      2400
   macro avg       0.78      0.77      0.77      2400
weighted avg       0.87      0.87      0.87      2400



## Проверим баланс классов

In [67]:
print("All :\n1 - ", sum(dat.bin), "\n0 - ", len(dat.bin) - sum(dat.bin), "\n", round(100*sum(dat.bin)/len(dat.bin), 2)) 
print("\nTrain :\n1 - ", sum(train_ans), "\n0 - ", len(train_ans) - sum(train_ans), "\n", round(100*sum(train_ans)/len(train_ans), 2)) 
print("\nTest :\n1 - ", sum(test_ans), "\n0 - ", len(test_ans) - sum(test_ans), "\n", round(100*sum(test_ans)/len(test_ans), 2) )
print("\nPrivat_ans :\n1 - ", sum(privat_ans), "\n0 - ", len(privat_ans) - sum(privat_ans), "\n", round(100*sum(privat_ans)/len(privat_ans), 2) ) 

All :
1 -  13194 
0 -  2806 
 82.46

Train :
1 -  8996 
0 -  1884 
 82.68

Test :
1 -  2226 
0 -  494 
 81.84

Privat_ans :
1 -  1972 
0 -  428 
 82.17


# Возьмем новый набор данных

In [68]:
dat2 = pd.read_json("Tools_and_Home_Improvement_5.json", lines = True)[["reviewText", "overall"]]

In [69]:
dat2["bin"] = dat2.overall > 3

## Истинное значение

In [70]:
rat = (len(dat2.bin)-sum(dat2.bin)) / sum(dat2.bin)
print("Соотношение false/true:",len(dat2.bin)-sum(dat2.bin), " / ", sum(dat2.bin))
print(accuracy_score(dat2.bin, search.best_estimator_.predict(tfidf_vect.transform(dat2.reviewText))))
print(classification_report(dat2.bin, search.best_estimator_.predict(tfidf_vect.transform(dat2.reviewText))))

Соотношение false/true: 20874  /  113602
0.8715161069633243
              precision    recall  f1-score   support

       False       0.63      0.43      0.51     20874
        True       0.90      0.95      0.93    113602

    accuracy                           0.87    134476
   macro avg       0.76      0.69      0.72    134476
weighted avg       0.86      0.87      0.86    134476



## Случайный выбор

In [71]:
df_rand_ch = dat2.sample(4000)
print("Соотношение false/true:",len(df_rand_ch.bin)-sum(df_rand_ch.bin), " / ", sum(df_rand_ch.bin))
print(accuracy_score(df_rand_ch.bin, search.best_estimator_.predict(tfidf_vect.transform(df_rand_ch.reviewText))))
print(classification_report(df_rand_ch.bin, search.best_estimator_.predict(tfidf_vect.transform(df_rand_ch.reviewText))))

Соотношение false/true: 630  /  3370
0.87025
              precision    recall  f1-score   support

       False       0.63      0.42      0.50       630
        True       0.90      0.96      0.93      3370

    accuracy                           0.87      4000
   macro avg       0.77      0.69      0.71      4000
weighted avg       0.86      0.87      0.86      4000



## Случайный равномерный выбор

In [72]:
dat2["Choice"] = 0
for i in range(len(dat2.bin)):
    dat2['Choice'][i] = (np.random.randint(round(10 / rat)) if dat2.bin[i] else np.random.randint(round(10)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:
new_test_dat = dat2.loc[dat2.Choice == 0][["reviewText", "overall", "bin", "Choice"]]
new_test_dat

,reviewText,overall,bin,Choice
45,These are replacements for the kitchen underca...,5,True,0
104,"Everyone loves this saw, or so it seems from t...",4,True,0
111,I bought this router shortly after it was intr...,1,False,0
123,I prefer brass. The Porter Cable guides are fi...,2,False,0
132,"This is a great collet, but I am giving it thr...",3,False,0
...,...,...,...,...
134367,These LEDs look just like any other incandesce...,3,False,0
134411,First words out of my mouth when I opened the ...,5,True,0
134444,I'm loving this but the handles are a little t...,4,True,0
134459,"I really like this, I recommend it",5,True,0


In [74]:
print("Соотношение false/true:",len(new_test_dat.bin)-sum(new_test_dat.bin), " / ", sum(new_test_dat.bin))
print("Всего " , (len(new_test_dat.bin)))
print(accuracy_score(new_test_dat.bin, search.best_estimator_.predict(tfidf_vect.transform(new_test_dat.reviewText))))
print(classification_report(new_test_dat.bin, search.best_estimator_.predict(tfidf_vect.transform(new_test_dat.reviewText))))

Соотношение false/true: 2098  /  2150
Всего  4248
0.698210922787194
              precision    recall  f1-score   support

       False       0.91      0.43      0.59      2098
        True       0.63      0.96      0.76      2150

    accuracy                           0.70      4248
   macro avg       0.77      0.70      0.67      4248
weighted avg       0.77      0.70      0.68      4248



## Случайный выбор с сохраненем баланса классов

In [75]:
df_rc_2 = pd.concat([dat2.loc[dat2['bin']].sample(round(4000*(1/(rat+1)))), dat2.loc[dat2['bin'] != True].sample(round(4000*rat/(rat+1)))], ignore_index=True)
print("Соотношение false/true:",len(df_rc_2.bin)-sum(df_rc_2.bin), " / ", sum(df_rc_2.bin))
print(accuracy_score(df_rc_2.bin, search.best_estimator_.predict(tfidf_vect.transform(df_rc_2.reviewText))))
print(classification_report(df_rc_2.bin, search.best_estimator_.predict(tfidf_vect.transform(df_rc_2.reviewText))))

Соотношение false/true: 621  /  3379
0.87025
              precision    recall  f1-score   support

       False       0.62      0.43      0.51       621
        True       0.90      0.95      0.93      3379

    accuracy                           0.87      4000
   macro avg       0.76      0.69      0.72      4000
weighted avg       0.86      0.87      0.86      4000



## По уверености классификатора

In [76]:
prob = search.best_estimator_.predict_proba(tfidf_vect.transform(dat2.reviewText))
prob = [x[0] for x in prob]

In [77]:
dat2["chance"] = prob

### Уверенность, что оценка <= 3

In [78]:
dat2 = dat2.sort_values(by = "chance")

In [79]:
dat2

,reviewText,overall,bin,Choice,chance
8726,perfect,5,True,20,6.189399e-09
27435,GREAT.. ........... .......... ............ .....,5,True,42,3.672248e-08
43376,Great thermostat,4,True,38,3.672248e-08
52596,great sharpener,5,True,14,3.672248e-08
50418,GREAT,5,True,27,3.672248e-08
...,...,...,...,...,...
18782,I unfortunately waited until the return period...,1,False,6,9.999336e-01
68774,poorly made. did not work more that 9 days. el...,1,False,5,9.999508e-01
28396,returned,1,False,4,9.999794e-01
31175,This seemed OK when I first got it. But it sto...,1,False,9,9.999931e-01


In [80]:
for i in range(len(dat2.Choice)):
    dat2.Choice[i] = (i % 32 == 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [81]:
new_test_dat_2 = dat2.loc[dat2.Choice]
new_test_dat_2

,reviewText,overall,bin,Choice,chance
91488,Let me make this simple:Pros:1. Excellent ligh...,4,True,True,0.000007
61600,Works great.,5,True,True,0.000020
130208,This is a great buy. Love the color and price.,5,True,True,0.000137
75424,This is great glue. Very nice bottle/dispenser...,5,True,True,0.000202
11744,My black and decker roto zip finally decided t...,5,True,True,0.000213
...,...,...,...,...,...
53536,"Maybe I used too much pressure, but the mandre...",2,False,True,0.995780
112992,Even in bright sunlight the bulb does not shut...,1,False,True,0.996675
75968,"I loved the flexibility and length however, in...",2,False,True,0.998521
58432,"Not worth of money, I had to replace 3 Ac adap...",1,False,True,0.998603


In [82]:
print("Соотношение false/true:",len(new_test_dat_2.bin)-sum(new_test_dat_2.bin), " / ", sum(new_test_dat_2.bin))
print("Всего " , (len(new_test_dat_2.bin)))

Соотношение false/true: 639  /  3564
Всего  4203


In [83]:
print(accuracy_score(new_test_dat_2.bin, search.best_estimator_.predict(tfidf_vect.transform(new_test_dat_2.reviewText))))
print(classification_report(new_test_dat_2.bin, search.best_estimator_.predict(tfidf_vect.transform(new_test_dat_2.reviewText))))

0.8722341184867951
              precision    recall  f1-score   support

       False       0.62      0.41      0.50       639
        True       0.90      0.95      0.93      3564

    accuracy                           0.87      4203
   macro avg       0.76      0.68      0.71      4203
weighted avg       0.86      0.87      0.86      4203

